<a href="https://colab.research.google.com/github/ffatmanurggultekin/scoliosis-detection/blob/main/Skolyoz_svm_hibrit_mat_dosyas%C4%B1_olusturma_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201  # ResNet50 yerine DenseNet201 kullanılıyor
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from google.colab import drive

# ✅ Google Drive'ı bağla
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ✅ Veri yolları
train_dir = '/content/drive/MyDrive/skolyoz_imadjust/train'
test_dir = '/content/drive/MyDrive/skolyoz_imadjust/test'

# Ensure TensorFlow uses CPU only
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
!pip install tensorflow==2.13.0

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [ ]:
import tensorflow as tf
print(tf.__version__)


2.13.0


In [ ]:


# ✅ Veri dosyalarını toplama fonksiyonu
def gather_paths_and_labels(directory):
    data = []
    for label, class_name in enumerate(['Normal', 'Scol']):
        class_dir = os.path.join(directory, class_name)
        for file in os.listdir(class_dir):
            data.append((os.path.join(class_dir, file), label))
    return np.array(data)

# ✅ Verileri yükleme
train_data = gather_paths_and_labels(train_dir)
test_data = gather_paths_and_labels(test_dir)

# ✅ Class Weights'i manuel olarak ayarla
class_weights_dict = {0: 1.0, 1: 0.7}  # Normal: 1.0, Scol: 0.7
print(f"\nUpdated Class Weights: {class_weights_dict}")

# ✅ Veri artırma (Data Augmentation) ayarları
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,           # Döndürme
    width_shift_range=0.2,       # Yatay kaydırma
    height_shift_range=0.2,      # Dikey kaydırma
    zoom_range=[0.8, 1.2],       # Yakınlaştırma
    horizontal_flip=True        # Ayna çevirme
)

val_test_datagen = ImageDataGenerator(rescale=1./255)  # Sadece normalizasyon

# ✅ Model oluşturma fonksiyonu (Fine-Tuning ile DenseNet201)
def create_feature_extraction_model():
    base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # İlk katmanları dondur, son 10 katmanı aç
    for layer in base_model.layers[:-10]:
        layer.trainable = False
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    x = layers.GlobalAveragePooling2D()(base_model.output)
    feature_extraction_model = models.Model(inputs=base_model.input, outputs=x)

    return feature_extraction_model


def extract_features(data, model, batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255)

    df = pd.DataFrame({'filename': data[:, 0], 'class': data[:, 1]})

    generator = datagen.flow_from_dataframe(
        df,
        x_col='filename', y_col='class',
        target_size=(224, 224), batch_size=batch_size, class_mode='binary', shuffle=False
    )

    features = model.predict(generator, steps=np.ceil(len(data) / batch_size))
    labels = df['class'].values

    return features, labels

# Create the feature extraction model
feature_extraction_model = create_feature_extraction_model()

# Extract features from the training and test data
train_features, train_labels = extract_features(train_data, feature_extraction_model)
test_features, test_labels = extract_features(test_data, feature_extraction_model)




Updated Class Weights: {0: 1.0, 1: 0.7}
74836368/74836368 [==============================] - 0s 0us/step
Found 180 validated image filenames belonging to 2 classes.
6/6 [==============================] - 10s 1s/step
Found 78 validated image filenames belonging to 2 classes.
3/3 [==============================] - 10s 5s/step


In [ ]:
import scipy.io
import os

# Kaydedilecek dosyanın yolu
save_path = "/content/drive/MyDrive/Skolyoz_calismalari/features_data_imadjust.mat"

# Kaydedilecek veriler
data_to_save = {
    "train_features": train_features,
    "train_labels": train_labels,
    "test_features": test_features,
    "test_labels": test_labels
}

# .mat dosyasını kaydet
scipy.io.savemat(save_path, data_to_save)

print(f"Dosya başarıyla kaydedildi: {save_path}")



Dosya başarıyla kaydedildi: /content/drive/MyDrive/Skolyoz_calismalari/features_data_imadjust.mat
